In [ ]:
%matplotlib inline

# Entrenando a un Clasificador CNN básico

Ha visto cómo definir redes neuronales, calcular pérdidas y hacer
actualizaciones de los pesos de la red.

Ahora podrías estar pensando,

## ¿Qué pasa con los datos?

Generalmente, cuando tiene que lidiar con datos de imagen, texto, audio o video,
puede usar paquetes estándar de python que cargan datos en una matriz numpy.
Entonces puedes convertir esta matriz en un ``torch.Tensor``.

- Para imágenes, paquetes como Pillow, OpenCV son útiles
- Para audio, paquetes como scipy y librosa
- Para texto, carga basada en Python o Cython sin procesar, o NLTK y
   SpaCy son útiles

Específicamente para la visión, se ha creado un paquete llamado
``torchvision``, que tiene cargadores de datos (dataloaders) para conjuntos de datos (datasets) comunes como
ImageNet, CIFAR10, MNIST, etc. y transformadores de datos para imágenes, a saber, ``torchvision.datasets`` y ``torch.utils.data.DataLoader``.

Esto proporciona una gran comodidad y evita escribir código repetitivo.

Para este tutorial, utilizaremos el conjunto de datos CIFAR10.
Tiene las clases: 'avión', 'automóvil', 'pájaro', 'gato', 'venado',
'perro', 'rana', 'caballo', 'barco', 'camión'. Las imágenes en CIFAR-10 son de
tamaño 3x32x32, es decir, imágenes en color de 3 canales de 32x32 píxeles de tamaño.

![](assets/cifar10.png)


## Pasos

Realizaremos los siguientes pasos en orden:

1. Cargue y normalice los conjuntos de datos de entrenamiento y prueba de CIFAR10 usando
   ``torchvision``
2. Definir una red neuronal convolucional
3. Defina una función de pérdida
4. Entrene a la red en los datos de entrenamiento
5. Pruebe la red con los datos de prueba

### 1. Cargar y normalizar CIFAR10

Usando ``torchvision``, es extremadamente fácil cargar CIFAR10.

In [ ]:
import torch
import torchvision
import torchvision.transforms as transforms

La salida de los conjuntos de datos de torchvision son imágenes PILImage de rango [0, 1].
Los transformamos a Tensores de rango normalizado [-1, 1].

<div class="alert alert-info"><h4>Nota</h4><p>Si se ejecuta en Windows y obtiene un BrokenPipeError, intente configurar
     el num_worker de torch.utils.data.DataLoader() a 0.</p></div>

In [ ]:
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

batch_size = 4

trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size,
                                          shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size,
                                         shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

Vamos a mostrar algunas de las imágenes de entrenamiento, por diversión.

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# functions to show an image


def imshow(img):
    img = img / 2 + 0.5     # unnormalize
    npimg = img.numpy()
    plt.imshow(np.transpose(npimg, (1, 2, 0)))
    plt.show()


# get some random training images
dataiter = iter(trainloader)
images, labels = dataiter.next()

# show images
imshow(torchvision.utils.make_grid(images))
# print labels
print(' '.join(f'{classes[labels[j]]:5s}' for j in range(batch_size)))

### 2. Definir una red neuronal convolucional
Copie la red neuronal de la sección Redes neuronales anterior y modifíquela para
tomar imágenes de 3 canales (en lugar de imágenes de 1 canal como se definió).



In [ ]:
import torch.nn as nn
import torch.nn.functional as F


class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = torch.flatten(x, 1) # flatten all dimensions except batch
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x


net = Net()

### 3. Definir una función de pérdida y un optimizador
Usemos una pérdida de entropía cruzada de clasificación y SGD con momentum.



In [ ]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

### 4. Entrenar a la red

Esto es cuando las cosas comienzan a ponerse interesantes.
Simplemente tenemos que recorrer nuestro iterador de datos y alimentar las entradas al
red y optimizar.



In [ ]:
for epoch in range(2):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 2000 == 1999:    # print every 2000 mini-batches
            print(f'[{epoch + 1}, {i + 1:5d}] loss: {running_loss / 2000:.3f}')
            running_loss = 0.0

print('Finished Training')

Guardemos rápidamente nuestro modelo entrenado:



In [ ]:
PATH = './cifar_net.pth'
torch.save(net.state_dict(), PATH)

Ver [aquí](https://pytorch.org/docs/stable/notes/serialization.html)
para obtener más detalles sobre cómo guardar modelos de PyTorch.

### 5. Pruebe la red con los datos de prueba

Hemos entrenado la red para 2 pases sobre el conjunto de datos de entrenamiento.
Pero necesitamos verificar si la red ha aprendido algo.

Verificaremos esto prediciendo la etiqueta de clase que la red neuronal
salidas, y comprobarlo contra la verdad de tierra. Si la predicción es
correcto, agregamos la muestra a la lista de predicciones correctas.

Bien, primer paso. Mostremos una imagen del conjunto de prueba para familiarizarnos.

In [ ]:
dataiter = iter(testloader)
images, labels = dataiter.next()

# print images
imshow(torchvision.utils.make_grid(images))
print('GroundTruth: ', ' '.join(f'{classes[labels[j]]:5s}' for j in range(4)))

A continuación, volvamos a cargar en nuestro modelo guardado (nota: guardar y volver a cargar el modelo no fue necesario aquí, solo lo hicimos para ilustrar cómo hacerlo):

In [ ]:
net = Net()
net.load_state_dict(torch.load(PATH))

Bien, ahora veamos qué piensa la red neuronal de estos ejemplos anteriores:

In [ ]:
outputs = net(images)

Las salidas son energías para las 10 clases.
Cuanto mayor sea la energía para una clase, mayor será la red
piensa que la imagen es de la clase particular.
Entonces, obtengamos el índice del score más alto:



In [ ]:
_, predicted = torch.max(outputs, 1)

print('Predicted: ', ' '.join(f'{classes[predicted[j]]:5s}'
                              for j in range(4)))

Los resultados parecen bastante buenos.

Veamos cómo funciona la red en todo el conjunto de datos.

In [ ]:
correct = 0
total = 0
# since we're not training, we don't need to calculate the gradients for our outputs
with torch.no_grad():
    for data in testloader:
        images, labels = data
        # calculate outputs by running images through the network
        outputs = net(images)
        # the class with the highest energy is what we choose as prediction
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy of the network on the 10000 test images: {100 * correct // total} %')

Eso se ve mucho mejor que el azar, que es un 10% de precisión (elegir una clase al azar entre 10 clases).
Parece que la red aprendió algo.

Hmmm, ¿cuáles son las clases que funcionaron bien y las clases que no funcionaron bien?

In [ ]:
# prepare to count predictions for each class
correct_pred = {classname: 0 for classname in classes}
total_pred = {classname: 0 for classname in classes}

# again no gradients needed
with torch.no_grad():
    for data in testloader:
        images, labels = data
        outputs = net(images)
        _, predictions = torch.max(outputs, 1)
        # collect the correct predictions for each class
        for label, prediction in zip(labels, predictions):
            if label == prediction:
                correct_pred[classes[label]] += 1
            total_pred[classes[label]] += 1


# print accuracy for each class
for classname, correct_count in correct_pred.items():
    accuracy = 100 * float(correct_count) / total_pred[classname]
    print(f'Accuracy for class: {classname:5s} is {accuracy:.1f} %')

Bien, ¿y ahora qué?

¿Cómo ejecutamos estas redes neuronales en la GPU?

## Entrenamiento en GPU
Al igual que transfiere un tensor a la GPU, transfiere la red neuronal a la GPU.

Primero definamos nuestro dispositivo como el primer dispositivo cuda visible si tenemos
CUDA disponible:

In [ ]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

# Assuming that we are on a CUDA machine, this should print a CUDA device:

print(device)

El resto de esta sección asume que ``dispositivo`` es un dispositivo CUDA.

Luego, estos métodos revisarán recursivamente todos los módulos y convertirán sus
parámetros y buffers a los tensores CUDA:

```python
net.to(dispositivo)
```


Recuerda que tendrás que enviar las entradas y los objetivos en cada paso
a la GPU también:

```python
entradas, etiquetas = datos[0].to(dispositivo), datos[1].to(dispositivo)
```

¿Por qué no noto una aceleración MASIVA en comparación con la CPU? porque tu red
es realmente pequeño

**Ejercicio:** Intente aumentar el ancho de su red (argumento 2 de
el primer ``nn.Conv2d``, y el argumento 1 del segundo ``nn.Conv2d`` –
deben ser el mismo número), vea qué tipo de aceleración obtiene.

**Metas cumplidas**:

- Comprender la biblioteca PyTorch y las redes neuronales a un alto nivel.
- Entrena una pequeña red neuronal para clasificar imágenes

## ¿Adónde voy después?

- `Entrenar una red ResNet de última generación en imagenet`
- `Entrena un generador de caras usando Generative Adversarial Networks`
- `Entrenar un modelo de lenguaje a nivel de palabra usando redes LSTM recurrentes`
- `Más ejemplos`
- `Más tutoriales`
- `Discutir PyTorch en los foros`
- `Chatea con otros usuarios en Slack`

In [ ]:
del dataiter